# Pretraining both network parts seperatly

In [1]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import torch
import matplotlib.pyplot as plt

In [2]:
# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

In [3]:
print("--- Pre-training tail model ---\n")

tail = TailLinear(label_encoded=True)
trainer = Trainer(nb_epochs=5, run='pretrained_seperatly_tailTraining')
trainer.fit(tail, dl_train_all, dl_val_all)

print("\n--- Pre-train classification model ---\n")

le_net = LeNet()
trainer = Trainer(nb_epochs=20, run='pretrained_seperatly_headTraining')
trainer.fit(le_net, dl_train_class, dl_val_class)

# Putting them together
siamese = Siamese(auxiliary=le_net, target=tail, weight_aux=0.3, softmax=False)
# trainer = Trainer(nb_epochs=5)
# trainer.fit(siamese, dl_train_all, dl_val_all)

print("\n--- Testing model ---\n")
trainer.test(le_net, dl_test_class, return_acc=False)
trainer.test(siamese, dl_test_all, return_acc=False)


--- Pre-training tail model ---

# Epoch 1/5:	 loss=0.67	 loss_val=0.61	 acc_val=77.68
# Epoch 2/5:	 loss=0.54	 loss_val=0.39	 acc_val=95.09
# Epoch 3/5:	 loss=0.3	 loss_val=0.17	 acc_val=100.0
# Epoch 4/5:	 loss=0.13	 loss_val=0.08	 acc_val=100.0
# Epoch 5/5:	 loss=0.06	 loss_val=0.04	 acc_val=100.0

--- Pre-train classification model ---

# Epoch 1/20:	 loss=1.65	 loss_val=0.67	 acc_val=84.13
# Epoch 2/20:	 loss=0.48	 loss_val=0.3	 acc_val=91.35
# Epoch 3/20:	 loss=0.31	 loss_val=0.2	 acc_val=94.23
# Epoch 4/20:	 loss=0.22	 loss_val=0.17	 acc_val=94.95
# Epoch 5/20:	 loss=0.18	 loss_val=0.15	 acc_val=95.19
# Epoch 6/20:	 loss=0.13	 loss_val=0.14	 acc_val=94.71
# Epoch 7/20:	 loss=0.11	 loss_val=0.13	 acc_val=95.67
# Epoch 8/20:	 loss=0.09	 loss_val=0.11	 acc_val=96.15
# Epoch 9/20:	 loss=0.07	 loss_val=0.1	 acc_val=96.39
# Epoch 10/20:	 loss=0.05	 loss_val=0.09	 acc_val=97.36
# Epoch 11/20:	 loss=0.05	 loss_val=0.15	 acc_val=94.23
# Epoch 12/20:	 loss=0.05	 loss_val=0.09	 acc_val=96.

In [4]:
# Do the same, but 10 times
results = []
for i in range(10):
    print(f"Run {i+1}/10")

    # Load data with class and larget than as targets
    dl_train_all, dl_val_all, dl_test_all = load_all_data()

    # Load data with class as target
    dl_train_class, dl_val_class, dl_test_class = load_class_data()

    # Training tail model
    tail = TailLinear(label_encoded=True)
    trainer = Trainer(nb_epochs=5, run='pretrained_seperatly_tailTraining')
    trainer.fit(tail, dl_train_all, dl_val_all, verbose=False)

    # Training leNet
    le_net = LeNet()
    trainer = Trainer(nb_epochs=20, run='pretrained_seperatly_headTraining')
    trainer.fit(le_net, dl_train_class, dl_val_class, verbose=False)

    # Putting both together
    siamese = Siamese(le_net, tail, softmax=False)

    # Testing
    acc = trainer.test(siamese, dl_test_all, test_verbose=True, return_acc=True)
    results.extend([acc])

Run 1/10
loss_test=0.14	 acc_test=97.36
Run 2/10
loss_test=0.14	 acc_test=97.17
Run 3/10
loss_test=0.13	 acc_test=97.95
Run 4/10
loss_test=0.12	 acc_test=97.17
Run 5/10
loss_test=0.16	 acc_test=97.17
Run 6/10
loss_test=0.17	 acc_test=96.68
Run 7/10
loss_test=0.12	 acc_test=98.14
Run 8/10
loss_test=0.17	 acc_test=96.88
Run 9/10
loss_test=0.14	 acc_test=97.07
Run 10/10
loss_test=0.13	 acc_test=97.75


In [7]:
# Print results
results = torch.Tensor(results)
print("Average accuracy:   {:.2f}".format(results.mean()))
print("Standard deviation: {:.2f}".format(results.std()))

Average accuracy:   96.97
Standard deviation: 0.58
